In [ ]:
# convert families to taxids
taxids = []
for family in mgv_families[1:]:
    taxid = ncbi.get_name_translator([family])
    taxids.append(taxid)

In [1]:
# Biopython to parse fasta files
from Bio import SeqIO
# pandas is for dataframes
import pandas as pd
# ete3 to convert from lineage to taxid (and vice versa)
from ete3 import NCBITaxa
ncbi = NCBITaxa()
ncbi.update_taxonomy_database()

NCBI database not present yet (first time used?)
Done. Parsing...


Loading node names...
2426563 names loaded.
279071 synonyms loaded.
Loading nodes...
2426563 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /Users/margotlavitt/.etetoolkit/taxa.sqlite ...
 2426000 generating entries... 

Inserting synonyms:      20000 


Uploading to /Users/margotlavitt/.etetoolkit/taxa.sqlite



Inserting taxid merges:  35000  

...

Loading node names...
2426563 names loaded.
279071 synonyms loaded.
Loading nodes...
2426563 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /Users/margotlavitt/.etetoolkit/taxa.sqlite ...
 2426000 generating entries...  generating entries... 
Uploading to /Users/margotlavitt/.etetoolkit/taxa.sqlite


Inserting synonyms:      20000 

Inserting taxid merges:  45000  

Inserting taxids:       25000  

Inserting taxids:       2425000 

In [2]:
# Download metadata file
# shell command
!curl -o  /Users/margotlavitt/resources/virusdb/mgv_metadata.tsv https://portal.nersc.gov/MGV/MGV_v1.0_2021_07_08/mgv_contig_info.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.2M  100 21.2M    0     0  11.6M      0  0:00:01  0:00:01 --:--:-- 11.6M


In [50]:
### convert each taxid in metadata to a lineage ###
# load mgv metadata
mgv_metadata= pd.read_csv('/Users/margotlavitt/resources/virusdb/mgv_metadata.tsv',
sep='\t')
mgv_metadata['ictv_family'] = mgv_metadata.apply(lambda x: x.ictv_family if x.ictv_family != 'crAss-phage' else 'uncultured crAssphage', axis = 1)
mgv_families = list(set(mgv_metadata['ictv_family']))

In [51]:
# convert families to taxids
taxids = []
for family in mgv_families[1:]:
    taxid = ncbi.get_name_translator([family])
    taxids.append(taxid)

In [52]:
# flatten list of dictionaries
taxids_dict = {}
for dictionary in taxids:
    taxids_dict.update(dictionary)

In [53]:
mgv_metadata['taxid'] = mgv_metadata['ictv_family'].map(taxids_dict)

In [56]:
#parse through fasta file
contig_to_taxid = mgv_metadata.set_index('contig_id').to_dict()['taxid']
new_mgv_sequences = []
for record in SeqIO.parse('/Users/margotlavitt/resources/virusdb/combined.fasta', "fasta"):
    updated_id = str(contig_to_taxid[record.id]).removeprefix('[').removesuffix(']')
    record.id = str(record.id) + '|kraken:taxid|' + updated_id
    new_mgv_sequences.append(record)

In [57]:
SeqIO.write(new_mgv_sequences, "/Users/margotlavitt/resources/virusdb/kraken_formatted_mgv.fasta", "fasta")

54118